# Data Playground

In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Do this to enable importing modules
src_path = os.path.join(os.path.abspath(""), "..")
sys.path.insert(0, src_path)

## Scraping tests

In [3]:
from data_ingestion_pipeline.crawlers.coin_telegraph import CoinTelegraphCrawler

Init data ingestion pipeline /Users/camiloramirezf/Documents/mai-repos/text-fetch-etl/data_ingestion_pipeline
Connection to database with uri: mongodb://localhost:30001,localhost:30002,localhost:30003/?replicaSet=my-replica-set successful


In [ ]:
from selenium import webdriver
from tempfile import mkdtemp
options = webdriver.ChromeOptions()
options.binary_location = "/opt/chrome/chrome"
options.add_argument("--no-sandbox")
options.add_argument("--headless=new")
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--log-level=3")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-notifications")
options.add_argument("--disable-dev-tools")
options.add_argument("--ignore-certificate-errors")
options.add_argument("--no-zygote")
options.add_argument(f"--user-data-dir={mkdtemp()}")
options.add_argument(f"--data-path={mkdtemp()}")
options.add_argument(f"--disk-cache-dir={mkdtemp()}")
options.add_argument("--remote-debugging-port=9222")

scroll_limit = 5

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.get("https://cointelegraph.com/")

In [ ]:
# crawler = CoinTelegraphCrawler()
# crawler.extract("")

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("https://cointelegraph.com/")

# Create an ActionChains object
actions = ActionChains(driver)

In [ ]:
main_carousel = driver.find_element(By.XPATH, "//div[@data-testid='carousel-main']")
first_li = main_carousel.find_element(By.XPATH, "//a[@data-testid='main-news-controls__link']")
first_li.click()

In [ ]:
first_li.is_displayed()

True

In [ ]:
from bs4 import BeautifulSoup
import time
scrolls = 0
max_scrolls = 20
last_height = 0

while True:
    # # Scroll down the page a few times
    driver.execute_script("window.scrollBy(0, 2 * window.innerHeight);")

    # Wait to load page
    time.sleep(2)

    # Break the loop if no more new content
    # new_height = driver.execute_script("return document.body.scrollHeight")
    # if new_height == last_height:
    #     break
    # last_height = new_height

    scrolls += 1
    if scrolls == max_scrolls:
        break

In [ ]:
articles = []

# Parse the page source
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find all the 'article' elements
article_elements = soup.find_all('article')

# For each 'article', find the 'h1' element and all 'div' siblings
for article in article_elements:
    h1_element = article.find('h1')
    title = h1_element.text
    summary = h1_element.find_next_sibling('div').text
    content_div = article.find('div', class_='post__content-wrapper')
    time_element = article.find('time')
    datetime = time_element.get('datetime') if time_element else None
    ps = content_div.find_all('p')
    content = ' '.join(p.text for p in ps if "Advertisement" not in p.text)

    # Create a dictionary for each article and append it to the list
    articles.append({'title': title, 'summary': summary, 'content': content, 'datetime': datetime})

# Close the driver
# driver.quit()

In [ ]:
import textwrap
print(textwrap.fill(articles[7]['title'], width=100))

 Weekend Wrap: SOL set for bronze, Bitcoiner fat fingers $100K fee and more


In [ ]:
len(articles)

12

## Mongo Writing and reading test


In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Do this to enable importing modules
src_path = os.path.join(os.path.abspath(""), "..")
sys.path.insert(0, src_path)
from data_ingestion_pipeline.models.documents import ArticleDocument
from data_ingestion_pipeline.crawlers.coin_telegraph import CoinTelegraphCrawler

Connection to database with uri: mongodb://localhost:30001,localhost:30002/?replicaSet=my-replica-set successful


### Write to local mongo replica

Use a test db and a test collection to write


In [ ]:
from pymongo import MongoClient
mongodb_uri = "mongodb://localhost:30001,localhost:30002,localhost:30003/?replicaSet=my-replica-set"
database_name = "db_test"
collection_name = "test"
test_data = {"name": "Copy dino"}

client = MongoClient(mongodb_uri)
db = client[database_name]
collection = db[collection_name]

try:
    result = collection.insert_one(test_data)
    print(f"Data inserted with _id: {result.inserted_id}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    client.close()

INFO:pymongo.serverSelection:{"message": "Waiting for suitable server to become available", "selector": "<function writable_server_selector at 0x107f19ea0>", "operation": "insert", "topologyDescription": "<TopologyDescription id: 663dd4d6c7ab7bcef353bfd2, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('localhost', 30001) server_type: Unknown, rtt: None>, <ServerDescription ('localhost', 30002) server_type: Unknown, rtt: None>, <ServerDescription ('localhost', 30003) server_type: Unknown, rtt: None>]>", "clientId": {"$oid": "663dd4d6c7ab7bcef353bfd2"}, "remainingTimeMS": 29}


Data inserted with _id: 663dd4d6c7ab7bcef353bfd3


### Write using crawler and the model classes

In [ ]:
crawler = CoinTelegraphCrawler()
crawler.extract()

In [ ]:
# test saving articles
doc = ArticleDocument(
    source="test",
    title="Moon will show",
    summary="It is happening, everything is mooning",
    content="Moon is the only destiny now",
    published_at="2024-05-05"
)

# articles = [doc]
# crawler.save(articles)

In [ ]:
doc.model_dump(exclude="id")

{'published_at': '2024-05-08',
 'title': 'Moon will show',
 'content': 'Moon is the only destiny now',
 'summary': 'It is happening, everything is mooning',
 'date': '2024-05-05'}

### Reading 


In [ ]:
from pymongo import MongoClient

mongodb_uri = "mongodb://localhost:30001,localhost:30002,localhost:30003/?replicaSet=my-replica-set"
database_name = "crypto-articles"
collection_name = "articles"

client = MongoClient(mongodb_uri)
db = client[database_name]
collection = db[collection_name]

try:
    documents = collection.find({})
    for doc in documents:
        print(doc)
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    client.close()

{'_id': '872ef4a5-399f-45e3-b38a-7fd7557670b5', 'published_at': '2024-05-08', 'title': 'Moon soon', 'content': 'Moon is the only destiny now', 'summary': 'It is happening, everything is mooning', 'date': '2024-05-05'}
{'_id': '61f69029-ba68-436e-936c-c5287e67b8a0', 'published_at': '2024-05-08', 'title': 'Moon later', 'content': 'Moon is the only destiny now', 'summary': 'It is happening, everything is mooning', 'date': '2024-05-05'}
{'_id': 'b077e14e-0261-4d26-9f05-ce5e8e4c56d1', 'published_at': '2024-05-08', 'title': 'Moon now', 'content': 'Moon is the only destiny now', 'summary': 'It is happening, everything is mooning', 'date': '2024-05-05'}
{'_id': 'cfe359f5-028f-45f3-bff7-6e30a6cec9be', 'published_at': '2024-05-08', 'title': 'Moon now', 'content': 'Moon is the only destiny now', 'summary': 'It is happening, everything is mooning', 'date': '2024-05-05'}
{'_id': '4ceb0d85-35cd-4574-8afa-7a8cd2d038d9', 'published_at': '2024-05-08', 'title': 'Moon now', 'content': 'Moon is the only d

## Telegram backfill test

In [ ]:
from data_ingestion_pipeline.crawlers import TelegramChannelsCrawler
telegram_crawler = TelegramChannelsCrawler()
result_day = await telegram_crawler.extract_day("2024-05-09")
result_total = await telegram_crawler.extract()

INFO:telethon.network.mtprotosender:Connecting to 149.154.167.92:443/TcpFull...
INFO:telethon.network.mtprotosender:Connection to 149.154.167.92:443/TcpFull complete!
INFO:crawlers.telegram:Telegram client connected
INFO:telethon.network.mtprotosender:Disconnecting from 149.154.167.92:443/TcpFull...
INFO:telethon.network.mtprotosender:Disconnection from 149.154.167.92:443/TcpFull complete!
INFO:telethon.network.mtprotosender:Connecting to 149.154.167.92:443/TcpFull...
INFO:telethon.network.mtprotosender:Connection to 149.154.167.92:443/TcpFull complete!
INFO:crawlers.telegram:Telegram client connected
INFO:telethon.network.mtprotosender:Disconnecting from 149.154.167.92:443/TcpFull...
INFO:telethon.network.mtprotosender:Disconnection from 149.154.167.92:443/TcpFull complete!


In [ ]:
def to_df(result):
    import pandas as pd
    df =  pd.DataFrame.from_records([record.model_dump() for record in result])
    df["parsed_dt"] = pd.to_datetime(df["published_at"])
    return df


In [ ]:
day_df =  to_df(result_day)
total_df = to_df(result_total)

In [ ]:
day_df.describe()

,parsed_dt
count,12
mean,2024-05-09 00:00:00
min,2024-05-09 00:00:00
25%,2024-05-09 00:00:00
50%,2024-05-09 00:00:00
75%,2024-05-09 00:00:00
max,2024-05-09 00:00:00


In [ ]:
day_df.head()

,id,published_at,title,content,summary,parsed_dt
0,b157c707-4b10-45f1-98a5-e0eac367b600,2024-05-09,📌 House Slated To Vote Today On Controversial ...,📌 House Slated To Vote Today On Controversial ...,None,2024-05-09
1,9259c4dc-d38f-49a5-a29d-391931c4d253,2024-05-09,🚀 Tether Boosts Crypto Payments in Eastern Eur...,🚀 Tether Boosts Crypto Payments in Eastern Eur...,None,2024-05-09
2,daa77652-27db-40dd-80c5-c4e9b3b77fb8,2024-05-09,💥 Solana 10% APY - only for M2 customers,💥 Solana 10% APY - only for M2 customers! 💥\n\...,None,2024-05-09
3,64a31a35-4856-432e-905a-65f454559ced,2024-05-09,‼️ Coincheck Aims for Nasdaq: Crypto Exchange ...,‼️ Coincheck Aims for Nasdaq: Crypto Exchange ...,None,2024-05-09
4,2a4bac3e-f469-465a-8992-65a81a1f9ab2,2024-05-09,"Nigeria Pushes Back Against Binance, Claims CE...","Nigeria Pushes Back Against Binance, Claims CE...",None,2024-05-09


## NewsAPI Test

In [ ]:
from newsapi import NewsApiClient
from data_ingestion_pipeline.config import settings
from data_ingestion_pipeline.crawlers import NewsFetcher
from datetime import datetime, timedelta

date_format="%Y-%m-%d"
start_dt = datetime.now() - timedelta(days=1)
end_dt = datetime.now() - timedelta(days=1)

api_articles = NewsFetcher().extract(
    page=1,
    start_date=start_dt.strftime(date_format),
    end_date=end_dt.strftime(date_format)
)

In [ ]:
ArticleDocument.dump_json(api_articles)

In [ ]:
df["parsed_dt"].describe()

count                    100
mean     2024-05-05 00:14:24
min      2024-05-01 00:00:00
25%      2024-05-03 00:00:00
50%      2024-05-06 00:00:00
75%      2024-05-07 00:00:00
max      2024-05-08 00:00:00
Name: parsed_dt, dtype: object

In [ ]:
items = [{"id": 1, "color": "red"}, {"id": 1, "color": "black"}]

a, b= zip(*items)

In [ ]:
b

('color', 'color')